<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/analyze_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'Analyze IV Traces'



from IPython.core.pylabtools import figsize
def IV_analyisis(abf_recordings_df,protocol_aliases, post_stim_times=[(0.0165,.03),(0.100,0.120)], stats_to_split= [(0, 'I_peak'),  (0, 'V_stim'), (1, 'I_mean')],to_plot=False):
    'Calulates results from an IV protocol. The peak and mean are measured'
    'from windows defined in a list of tuples [(start,stop),...]. All values'
    'are written to the main dataframe under "IV_stats". This is a list of dicts'
    'each dict corresponds to one anaysis window reporting the time range, peak etc.'
    'Optional variable stats_to_split is used to pull specific measures from IV_stats'
    'and write them to their own column. Stats_to_split is a list of tuples, indicating'
    'which window and witch measure to write to the dataframe. [(window index,measure key)...]'


    correct_protocol = [ p in protocol_aliases for p in abf_recordings_df['protocol']]
    abf_recordings_df['IV_stats'] = None
    abf_recordings_df['IV_stats'] = abf_recordings_df['IV_stats'].astype(object)

    stats_to_split = ( (0, 'I_peak'),  (0, 'V_stim'), ( (1, 'I_mean')))
    for t in stats_to_split:
        name = t[1]
        abf_recordings_df['IV_'+name] = None
        abf_recordings_df['IV_'+name] = abf_recordings_df['IV_'+name].astype(object)

    for file_name in tqdm( abf_recordings_df.index[correct_protocol]):
        abf = pyabf.ABF( file_name )
        passing_sweeps=abf_recordings_df.at[file_name,'passing_sweeps']

        pass_rate, passing_sweeps, QC_check_df, QC_val_df = Vclamp_QC(file_name,to_plot=False,verbose=False,Vhold_range = 100)
        # display(QC_val_df)
        if to_plot: plot_sweeps_and_command(abf)
        # passing_sweeps = abf.sweepList # OVERRIDE
        print(passing_sweeps)
        abf = pyabf.ABF( file_name )
        if to_plot: print(file_name)
        IV_stats =analyze_IV(abf, passing_sweeps, post_stim_times,to_plot =to_plot)
        # [print(r) for r in IV_stats]
        
        abf_recordings_df.at[file_name,'IV_stats'] = IV_stats

        #### 
        if len(IV_stats) > 0:
            for t in stats_to_split:
                name = t[1]
                range = t[0]
                values = IV_stats[range][name]
                values = np.array(values).tolist()
                abf_recordings_df.at[file_name,'IV_'+name] = values

    return abf_recordings_df



def get_IV_measures(abf,passing_sweeps,post_stim_times,to_plot=False,spike_thresh=100000):
    'Takes the abf data and calculates the mean and peak for each window.'
    'Optionaly sweeps with APs can be filtered out using spike_threshold.'
    'Only sweeps passing the QC filter are used. Returns a list of dicts with'
    'each dict corresponding to one anaysis window reporting the time range, peak etc.'

    IV_stats = []
    theta, offset, correct_ch1 = predict_telegraph(abf)
    



    if len(passing_sweeps)==0:
        return IV_stats

    if to_plot:
        fig, axs = plt.subplots(len(post_stim_times),figsize = [12,4] )
        if len(post_stim_times)==1: axs = [axs]
    
    for t in range(len(post_stim_times)):
        window_stats = {}
        start = post_stim_times[t][0]
        stop = post_stim_times[t][1]
        delta_t = stop-start
        flank_neg = start - (delta_t)*0.5
        flank_pos = stop + (delta_t)*0.5
        full_plot_range_ind =   np.logical_and(  abf.sweepX>flank_neg, abf.sweepX<flank_pos )
        spikeless_sweeps = []
        for s in passing_sweeps:
            if to_plot: s_shift_t = delta_t*0.03*s
            if to_plot: s_shift_I = 100*s
            abf.setSweep(s)
            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]

            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]
            analysis_range = np.logical_and(  abf.sweepX>start, abf.sweepX<stop )
            time_analysis = abf.sweepX[analysis_range]
            I_analysis = movmean( abf.sweepY[analysis_range], 4)
            dIdt = movmean(np.diff(I_abrg, prepend=I_abrg[0]),1)

            if np.max(abs(dIdt))>spike_thresh:
                'indent holder'
                # if to_plot:axs[t].plot( time_abrg+s_shift_t, dIdt+s_shift_I,'r')
            else:
                # if to_plot:axs[t].plot( time_abrg+s_shift_t, dIdt+s_shift_I,'k')
                spikeless_sweeps.append(s)


        I_peak_l = []
        time_peak_l = []
        I_mean_l = []
        V_stim_l = []
        
        for s in  spikeless_sweeps :
            abf.setSweep(s)
            s_shift_t = delta_t*0.03*s * 0
            s_shift_I = 100*s * 0

            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]
            analysis_range = np.logical_and(  abf.sweepX>start, abf.sweepX<stop )
            time_analysis = abf.sweepX[analysis_range]
            I_analysis = abf.sweepY[analysis_range]
            

            # Peaks
            if to_plot: axs[t].plot( time_abrg+s_shift_t, I_abrg+s_shift_I,'k')
            
            I_mean = np.median( I_analysis )
            if to_plot: axs[t].plot( time_analysis+s_shift_t, (I_mean*np.ones_like(time_analysis))+s_shift_I,'r' )            
            
            local_I_mean = movmean(I_analysis,100)
            peak_ind = np.where(abs(I_analysis-I_mean) == np.max(abs(I_analysis-I_mean)))[0]
            # print(peak_ind)
            if len(peak_ind)>1: peak_ind=[np.min(peak_ind)]
                # print('drop')
                # print(peak_ind)

            I_peak =I_analysis[peak_ind]
            t_peak = time_analysis[peak_ind]
            if to_plot: axs[t].scatter( t_peak+s_shift_t, I_peak+s_shift_I, color='m' )

            abf.setSweep(sweepNumber=s, channel=1)
            command_trace = abf.sweepY


            I_peak_l.append(I_peak)
            time_peak_l.append(t_peak)
            I_mean_l.append(I_mean)
            # raw_v = np.median( abf.sweepC[analysis_range])
            # V_stim_l.append( 10*np.round( (raw_v +offset )/10))

            command_v = np.median( command_trace[analysis_range])
            V_stim_l.append(command_v)
    

        I_peak_l = np.concatenate(I_peak_l)
        time_peak_l = np.concatenate(time_peak_l)


        # print(I_peak_l)
        # print(time_peak_l)
        # print(I_mean_l)
        # print(V_stim_l)


        window_stats['range'] = (start,stop)
        window_stats['I_peak'] = I_peak_l
        window_stats['I_peak_time'] = time_peak_l
        window_stats['I_mean'] = I_mean_l
        window_stats['V_stim'] = np.stack(V_stim_l).flatten()
        IV_stats.append(window_stats)

    return IV_stats
    

def analyze_IV(abf, passing_sweeps, post_stim_times, to_plot=False):
    'This is largely a wrapper for get_IV_measures(). The main role'
    'of this function is to plot the summary data, ie the traditional IV plot'

    IV_stats = get_IV_measures(abf, passing_sweeps,post_stim_times,to_plot=to_plot,spike_thresh=1e6)
    any_sweeps = any(np.greater([len(w['V_stim']) for w in IV_stats],0))
    if not any_sweeps:
        return IV_stats

    if to_plot:
        fig, axs = plt.subplots(1,2,figsize = (9,4))
        early_peak = IV_stats[0]['I_peak']
        V_stim = IV_stats[0]['V_stim']
        axs[0].plot(V_stim,early_peak,color='m',marker='o',label='Peak Current')


        # ax.scatter(V_stim,early_peak,color='k')
        late_mean = IV_stats[1]['I_mean']
        V_stim = IV_stats[1]['V_stim']
        axs[0].plot(V_stim,late_mean,color='r',marker='o',label='Steady State Current')

        from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
        axs[0].xaxis.set_major_locator(MultipleLocator(20))
        axs[0].legend(loc="upper left",frameon=True,framealpha=1)
        axs[0].set_ylabel('Current (pA)')
        axs[0].set_xlabel('Membrane Potential (mV)')

        
        # ax.grid(True)
        axs[0].spines['top'].set_visible(False)
        axs[0].spines['right'].set_visible(False)
        axs[0].spines['bottom'].set_visible(False)
        axs[0].spines['left'].set_visible(False)
        yL = axs[0].get_ylim()
        xL = axs[0].get_xlim()
        axs[0].plot(xL,[0,0],':k')
        axs[0].plot([-70,-70],yL,':k')


        for s in passing_sweeps:
            abf.setSweep(s)
            axs[1].plot(abf.sweepX,abf.sweepC,'k')
        
        yL = axs[1].get_ylim()
        axs[1].axvspan( IV_stats[0]['range'][0],IV_stats[0]['range'][1],yL[0],yL[1], alpha=0.2 ,color='m')
        axs[1].axvspan( IV_stats[1]['range'][0],IV_stats[1]['range'][1],yL[0],yL[1], alpha=0.2 ,color='r')
        plt.show()
    return IV_stats